In [2]:
# reload modules in .py files
%load_ext autoreload
%autoreload 2

In [3]:
# pull repo
!git clone https://github.com/char-tan/sparsity

Cloning into 'sparsity'...
remote: Enumerating objects: 34, done.
remote: Counting objects: 100% (34/34), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 34 (delta 10), reused 23 (delta 5), pack-reused 0
Unpacking objects: 100% (34/34), done.


In [4]:
# change working directory
import os
os.chdir('sparsity/sparse_ops')

In [ ]:
# THIS DOESN'T WORK
# !python setup.py install
# Using -e has caveats: https://stackoverflow.com/a/20043907
!pip3 install .

### Sample usage of our extension

In [6]:
import torch
import os, sys
sys.path.append(os.getcwd())
import sparse_ops_cpp

a = torch.tensor([1])
b = a
c = sparse_ops_cpp.add(a, b)
print(c)

tensor([2])
